In [24]:
# Dependencies
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
import requests

# Migration inflow by authority

Information about the data can be fount here: https://www.nomisweb.co.uk/sources/census_2021_mig

In [25]:
# Create df for migration into England and Wales
mig_ltla_in_df = pd.read_csv("./Resources/MIG004EW_LTLA_IN.csv")
mig_ltla_in_df.head()


,Lower tier local authorities code,Lower tier local authorities label,Migration LTLA (inflow) (6 categories) code,Migration LTLA (inflow) (6 categories) label,Ethnic group (6 categories) code,Ethnic group (6 categories) label,National Statistics Socio-economic Classification (NS-SeC) (10 categories) code,National Statistics Socio-economic Classification (NS-SeC) (10 categories) label,Count
0,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,-8,Does not apply,0
1,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,1,"L1, L2 and L3: Higher managerial, administrati...",0
2,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,2,"L4, L5 and L6: Lower managerial, administrativ...",0
3,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,3,L7: Intermediate occupations,0
4,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,4,L8 and L9: Small employers and own account wor...,0


In [26]:
# Countlocal authority labels
in_authority = mig_ltla_in_df.groupby(['Lower tier local authorities code', 'Lower tier local authorities label', 'Migration LTLA (inflow) (6 categories) label'])['Count'].sum().reset_index()
in_authority = in_authority.sort_values(by='Count', ascending=False)

in_authority

,Lower tier local authorities code,Lower tier local authorities label,Migration LTLA (inflow) (6 categories) label,Count
1583,E08000025,Birmingham,Lived at the same address one year ago,1007979
1643,E08000035,Leeds,Lived at the same address one year ago,689265
293,E06000052,Cornwall,Lived at the same address one year ago,504531
1625,E08000032,Bradford,Lived at the same address one year ago,495205
341,E06000060,Buckinghamshire,Lived at the same address one year ago,491753
...,...,...,...,...
1728,E09000013,Hammersmith and Fulham,Does not apply,0
612,E07000077,Uttlesford,Does not apply,0
1086,E07000173,Gedling,Does not apply,0
618,E07000078,Cheltenham,Does not apply,0


In [27]:

migration_categories = in_authority['Migration LTLA (inflow) (6 categories) label'].unique()

# Create a separate bar graph for each category
for category in migration_categories:
    filtered_data = in_authority[in_authority['Migration LTLA (inflow) (6 categories) label'] == category]
    
    fig = px.bar(filtered_data, x='Lower tier local authorities label', y='Count',
                 title=f'Migration LTLA: {category}')
    
    fig.update_layout(xaxis_title='Local Authority', yaxis_title='Count')
    fig.update_xaxes(categoryorder='total ascending')  # Sort authorities by total count
    fig.show()

# API For Authority Areas

Can be found here: https://findthatpostcode.uk/#api

In [29]:
# Define a function to fetch GeoJSON data
def get_geojson(lower_tier_local_authority_code):
    base_url = "https://findthatpostcode.uk/areas/"
    geojson_url = f"{base_url}{lower_tier_local_authority_code}.geojson"
    try:
        response = requests.get(geojson_url)
        if response.status_code == 200:
            return response.json()  # Assuming the URL returns GeoJSON
    except Exception as e:
        print(f"Error fetching GeoJSON for {lower_tier_local_authority_code}: {str(e)}")
    return None

# Apply the function to create a new column with GeoJSON data
in_authority['GeoJSON'] = in_authority['Lower tier local authorities code'].apply(get_geojson)

In [30]:
in_authority

,Lower tier local authorities code,Lower tier local authorities label,Migration LTLA (inflow) (6 categories) label,Count,GeoJSON
1583,E08000025,Birmingham,Lived at the same address one year ago,1007979,{'features': [{'geometry': {'coordinates': [[[...
1643,E08000035,Leeds,Lived at the same address one year ago,689265,{'features': [{'geometry': {'coordinates': [[[...
293,E06000052,Cornwall,Lived at the same address one year ago,504531,{'features': [{'geometry': {'coordinates': [[[...
1625,E08000032,Bradford,Lived at the same address one year ago,495205,{'features': [{'geometry': {'coordinates': [[[...
341,E06000060,Buckinghamshire,Lived at the same address one year ago,491753,{'features': [{'geometry': {'coordinates': [[[...
...,...,...,...,...,...
1728,E09000013,Hammersmith and Fulham,Does not apply,0,{'features': [{'geometry': {'coordinates': [[[...
612,E07000077,Uttlesford,Does not apply,0,{'features': [{'geometry': {'coordinates': [[[...
1086,E07000173,Gedling,Does not apply,0,{'features': [{'geometry': {'coordinates': [[[...
618,E07000078,Cheltenham,Does not apply,0,{'features': [{'geometry': {'coordinates': [[[...
